# Decision Tree Algorithm from SCRATCH

### Dataset - For Traning

In [330]:
"""
Color   Diameter    Label

Green   3           Apple
Yellow  3           Apple
Red     1           Grape
Red     1           Grape
Yellow  3           Lemon
"""

'\nColor   Diameter    Label\n\nGreen   3           Apple\nYellow  3           Apple\nRed     1           Grape\nRed     1           Grape\nYellow  3           Lemon\n'

In [331]:
from __future__ import print_function

In [332]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

header = ["color", "diameter", "label"]

To Find the Unique Values for a column in the dataset

In [333]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

#Verification
unique_vals(training_data, 2)

{'Apple', 'Grape', 'Lemon'}

To count the no. of each type of example in the dataset

In [334]:
def class_counts(rows):
    counts = {} #dictionary
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts
"""
#Verification
class_counts(training_data)
"""

'\n#Verification\nclass_counts(training_data)\n'

To check if a data is numeric

In [335]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)
"""
#Verification
is_numeric("Red")
"""

'\n#Verification\nis_numeric("Red")\n'

Question: It will be used to split the dataset
Example: it is 1 record from the dataset, which has all the columns / single data

In [336]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))
"""
#Verification
q = Question(0, 'Red')
print(q)

example = training_data[1]
q.match(example)
"""

"\n#Verification\nq = Question(0, 'Red')\nprint(q)\n\nexample = training_data[1]\nq.match(example)\n"

Partition of the Database based on the Question

In [337]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows
"""
#Verification:
true_rows, false_rows = partition(training_data, Question(1, 3))
false_rows
"""

'\n#Verification:\ntrue_rows, false_rows = partition(training_data, Question(1, 3))\nfalse_rows\n'

To calculate Ginin Impurity for list of rows


Gini Index = 1 - summation [(Prob)^2]

In [338]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1 
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity
"""
#Verifcation:
pure = [['A'], ['A']]
equal_mix= ['A', 'B']
impure = ['A', 'B', 'C', 'D', 'E']
print(gini(pure))
print(gini(equal_mix))
print(gini(impure))
"""

"\n#Verifcation:\npure = [['A'], ['A']]\nequal_mix= ['A', 'B']\nimpure = ['A', 'B', 'C', 'D', 'E']\nprint(gini(pure))\nprint(gini(equal_mix))\nprint(gini(impure))\n"

To calculate Information Gain

In [339]:
def info_gain(left, right, current_uncertainity):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainity - p * gini(left) - (1-p) * gini(right)
"""
#Verification
current_uncertainity = gini(training_data)
current_uncertainity

true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainity)
"""


"\n#Verification\ncurrent_uncertainity = gini(training_data)\ncurrent_uncertainity\n\ntrue_rows, false_rows = partition(training_data, Question(0, 'Green'))\ninfo_gain(true_rows, false_rows, current_uncertainity)\n"

To find the best split by finding the best question using INFO_GAIN

In [340]:
def find_best_split(rows):
    best_gain = 0 
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question =  Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows)==0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainity)

            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

"""
#Verfication:
best_gain, best_question = find_best_split(training_data)
best_question
"""

'\n#Verfication:\nbest_gain, best_question = find_best_split(training_data)\nbest_question\n'

In [341]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [342]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [343]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [344]:
def print_tree(node, spacing = ""):
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    print(spacing + str(node.question))

    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "    ")

    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "   ")

In [345]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
    Is color == Yellow?
    --> True:
        Predict {'Apple': 1, 'Lemon': 1}
    --> False:
       Predict {'Apple': 1}
--> False:
   Predict {'Grape': 2}


In [346]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [347]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [348]:
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
    ]

In [349]:
for row in testing_data:
    print("Actual: %s & Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple & Predicted: {'Apple': '100%'}
Actual: Apple & Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape & Predicted: {'Grape': '100%'}
Actual: Grape & Predicted: {'Grape': '100%'}
Actual: Lemon & Predicted: {'Apple': '50%', 'Lemon': '50%'}
